# bloom_kserve
Deploy PyTorch model with TorchServe InferenceService

If you want to know more detailes, please refer to https://github.com/kserve/kserve/tree/master/docs/samples/v1beta1/torchserve/v1/bloom. 

This tutorial is that study the above example and improve it so that can run inferenceservice in the Kubeflow on vSphere environment.

## 1. Prepare MAR model package and it's config

Clone https://github.com/AmyHoney/bloom.git repository and follow the README.md and bloom_torchserve.ipynb for creating the MAR file including serialized model, other dependent files and configuration.

Study the example https://github.com/pytorch/serve/tree/master/examples/large_models/Huggingface_accelerate and improve it so that it can run in the kubeflow on vSphere environment
The above Torchserve example works on shard version of Huggingface models.
For sharding the Huggingface models you can use the following script, and then compress the model

**In this tutorial we will show how to serve Large Huggingface models(Bloom) with TorchServe on KServe.**



**Notice:** Install requirments from tsinghua source as below if your network without VPN when you run step 3: generate MAR file, Or got error: ImportError: cannot import name 'BloomForCausalLM' from 'transformers' and ModuleNotFoundError: No module named 'ts.torch_handler.custom_handler'

**Besides**, there use bloom-560m to do inferenceservice due to resources limit.

In [43]:
!cat requirements.txt

transformers==4.25.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
accelerate==0.18.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
pynvml==8.0.4 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
!ls

config	model-store  sample_text4.json	test.ipynb


In [2]:
# check have prepared the MAR model and it's config
!ls ./model-store/
!ls ./config/

bloom.mar
config.properties


In [11]:
!cat ./config/config.properties

inference_address=http://0.0.0.0:8085
management_address=http://0.0.0.0:8085
metrics_address=http://0.0.0.0:8082
grpc_inference_port=7070
grpc_management_port=7071
enable_metrics_api=true
metrics_format=prometheus
number_of_netty_threads=4
job_queue_size=10
enable_envvars_config=true
install_py_dep_per_model=true
model_store=/mnt/models/model-store
model_snapshot={"name":"startup.cfg","modelCount":1,"models":{"bloom":{"1.0":{"defaultVersion":true,"marName":"bloom.mar","minWorkers":1,"maxWorkers":5,"batchSize":1,"maxBatchDelay":5000,"responseTimeout":120}}}}

## 2. Create PVC and Mount the Model.

### 2.1 Create a volume in Kubeflow UI

Use your web browser to login to Kubeflow, and click volumes, then click the `New volume` in the top right bar to create pvc.

![voumes creating](./img/volumes.png)

### 2.2 Create pv-pod to mount the model


In [6]:
%%bash

cat << EOF | kubectl create -n zyajing -f -
apiVersion: v1
kind: Pod
metadata:
  name: model-store-pod
spec:
  volumes:
    - name: pv-storage
      persistentVolumeClaim:
        claimName: model-local-claim
  containers:
    - name: pv-container
      image: ubuntu
      command: [ "sleep" ]
      args: [ "infinity" ]
      volumeMounts:
        - mountPath: "/pv"
          name: pv-storage
      resources:
        limits:
          memory: "4Gi"
          cpu: "2"
EOF

pod/model-store-pod created


In [ ]:
# Move the config.properties and MAR file to PVC

!kubectl exec -it model-store-pod -n zyajing -- cd pv/ && mkdir model_store
!kubectl exec -it model-store-pod -n zyajing -- cd pv/ && mkdir config

In [ ]:
!kubectl exec -it model-store-pod -n zyajing -- ls -al /pv

In [ ]:
# start to copy
!kubectl cp bloom.mar model-store-pod:/pv/model-store/ -n zyajing
!kubectl cp config.properties model-store-pod:/pv/config/ -n zyajing

## 3. Create the InferenceService

**Notice:** Remember to delete `model-store-pod` pod to unmount pvc, Or if you create inferenceservice YAML file, will got `model-local-claim` pvc have been used.

In [9]:
%%bash

# Apply the CRD
# storageUri: "pvc://${PVC_NAME}/model.joblib"
cat << EOF | kubectl create -n zyajing -f -
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: "torchserve-bloom-560mm"
spec:
  predictor:
    pytorch:
      storageUri: pvc://model-local-claim
      resources:
          limits:
            cpu: 8
            memory: 16Gi
            nvidia.com/gpu: 1
EOF

inferenceservice.serving.kserve.io/torchserve-bloom-560mm created


In [ ]:
# Check logs 
# kubectl logs <pod_name> -n zyajing
kubectl logs torchserve-bloom-560m-predictor-default-00001-deployment-876shh -n zyajing

## 4. Test

### 4.1 check GPU resources

In [24]:
!kubectl get pods -A | grep dcgm

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:zyajing:default-editor" cannot list resource "pods" in API group "" at the cluster scope


In [ ]:
# If you run the below command, got Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:zyajing:default-editor" cannot list resource "pods" in API group "" at the cluster scope
# Run the below command in the terminal sucessfully.
$ kubectl get po -n zyajing -o wide
NAME                                                              READY   STATUS    RESTARTS   AGE     IP           NODE                                                     NOMINATED NODE   READINESS GATES
torchserve-bloom-560m-predictor-default-00001-deployment-876shh   3/3     Running   0          86m     192.0.7.55   liuqi-test-vgpu-v100-16c-2-np-2-8bfsx-f694667c-pc8wr     <none>           <none>

!kubectl get pods -A | grep dcgm
kubectl get pods -A -o wide| grep dcgm | grep pc8wr
gpu-operator                   nvidia-dcgm-exporter-9pk2w                                               1/1     Running            0                  4d18h   192.0.7.6     liuqi-test-vgpu-v100-16c-2-np-2-8bfsx-f694667c-pc8wr     <none>           <none>

$ kubectl exec -n gpu-operator nvidia-dcgm-exporter-9pk2w -- nvidia-smi
Defaulted container "nvidia-dcgm-exporter" out of: nvidia-dcgm-exporter, toolkit-validation (init)
Tue Apr 25 03:06:45 2023
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID V100-16C       On   | 00000000:02:00.0 Off |                    0 |
| N/A   N/A    P0    N/A /  N/A |   1834MiB / 16384MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+

# It seems that the gpu resources have been occupied.

## 3.2 Option 1 curl to access model

In [6]:
# !export MODEL_NAME=bloom
# !export ISVC_NAME=torchserve-bloom-560m 
!kubectl get inferenceservice torchserve-bloom-560m -n zyajing -o jsonpath='{.status.url}' | cut -d "/" -f 3

torchserve-bloom-560m.zyajing.example.com


Use your web browser to login to Kubeflow, and get Cookies: authservice_session (Chrome: Developer Tools -> Applications -> Cookies)

In [40]:
!export SERVICE_HOSTNAME='torchserve-bloom-560m.zyajing.example.com'
!export SESSION='MTY4MjM5Mzc1N3xOd3dBTkRWUldsRTNSMFpNVmxOR1ZqWk5XRE5GVkZNMVdVbEVOVW8yTnpaRlJscEpXazVhVkZvME4xQlFTek5IVWtaWlQwZEZTbEU9fEgO5cAxqnPfp9k9VVne7NGpI-7kHEKgYo0BqlnUFbtI'


In [39]:
!echo $SERVICE_HOSTNAME
!echo $SESSION

torchserve-bloom-560m.zyajing.example.com
MTY4MjMwNzMxMHxOd3dBTkVSUVFUWkpNMHBPV0ZWUVR6SlpXRE5XTmxSWU1rbFlSVXBQVlVjMFVVTXpXVVpTVjFCQ01rczFOelpXVEZwSVYweFdORkU9fJSm63ckzmJ0ZzAS1qpgi-NcY-tuPLBuK9CIWPd_DmP1


In [41]:
# The first step is refer https://kserve.github.io/website/0.10/get_started/first_isvc/#4-determine-the-ingress-ip-and-ports to determine the ingress IP and ports and set INGRESS_HOST：10.105.150.44 and INGRESS_PORT:80
!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: authservice_session=$SESSION" http://10.105.150.44:80/v1/models/bloom:predict -d @./sample_text4.json
# From the below log, there is no find Host and authservice_session, even though I have exported Host and authservice_session before

*   Trying 10.105.150.44:80...
* TCP_NODELAY set
* Connected to 10.105.150.44 (10.105.150.44) port 80 (#0)
> POST /v1/models/bloom:predict HTTP/1.1
> Host: 
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=
> Content-Length: 61
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 61 out of 61 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 403 Forbidden
< date: Tue, 25 Apr 2023 04:11:09 GMT
< server: envoy
< content-length: 0
< x-envoy-upstream-service-time: 4
< 
* Connection #0 to host 10.105.150.44 left intact


In [10]:
# use "-d @./sample_text.json" to curl, Or if you "-d ./sample_text.txt", will got error "400: Unrecognized request format: Expecting value: line 1 column 1 (char 0)"
!curl -v -H "Host: torchserve-bloom-560m.zyajing.example.com" -H "Cookie: authservice_session=MTY4MjM5Mzc1N3xOd3dBTkRWUldsRTNSMFpNVmxOR1ZqWk5XRE5GVkZNMVdVbEVOVW8yTnpaRlJscEpXazVhVkZvME4xQlFTek5IVWtaWlQwZEZTbEU9fEgO5cAxqnPfp9k9VVne7NGpI-7kHEKgYo0BqlnUFbtI" http://10.105.150.44:80/v1/models/bloom:predict -d @./sample_text.json 

*   Trying 10.105.150.44:80...
* TCP_NODELAY set
* Connected to 10.105.150.44 (10.105.150.44) port 80 (#0)
> POST /v1/models/bloom:predict HTTP/1.1
> Host: torchserve-bloom-560m.zyajing.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTY4MjM5Mzc1N3xOd3dBTkRWUldsRTNSMFpNVmxOR1ZqWk5XRE5GVkZNMVdVbEVOVW8yTnpaRlJscEpXazVhVkZvME4xQlFTek5IVWtaWlQwZEZTbEU9fEgO5cAxqnPfp9k9VVne7NGpI-7kHEKgYo0BqlnUFbtI
> Content-Length: 61
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 61 out of 61 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 372
< content-type: application/json; charset=UTF-8
< date: Wed, 26 Apr 2023 03:14:19 GMT
< server: envoy
< x-envoy-upstream-service-time: 2970
< 
* Connection #0 to host 10.105.150.44 left intact
{"predictions": ["My dog is cute and smart and kind. He loves to be around and will be an easy companion!\nCotton is the perfect material for a wall with a wooden base. It ca